# **Analyse de sentiments sur les films du TOP 100 box office (Notebook pour le processing des données) 2/3**
**Projet Python - 2A ENSAE**

AUMONT Louis-Armand, KHAIRALDIN Ahmed, GIMENES Vincent

## Introduction

L'objectif de ce Notebook va être de charger la base de données, puis nettoyer les commentaires pour les "tokenisés" et enfin les "lemmatisés"
et appliquer des algorithmes de NLP sur nos commentaires "lemmatisés" afin de déduire pour chaque commentaire un score de polarité et afin former un nouveau dataset final où chaque observation sera un film.

Avant de pouvoir faire de l'analyse de sentiment pour chaque commentaire, il faut d'abord les nettoyer pour les rendre lisibles pour l'ordinateur. Premièrement, nous allons transformer le fichier data_reviews en une base de données où chaque observation est un commentaire. Puis, nous allons nettoyer chaque commentaire en utilisant des fonctions permettant de supprimer la ponctuation, les stop-words et la lemmatisation.



####  Download Library ⚙️

In [9]:
!pip install spacy
!pip install wordcloud
!pip install textblob
!pip install nltk

Il faut également télécharger une fois ces sortes de sous packages:

In [10]:
# Téléchargement d'éléments nécessaires à la tokenisation
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/onyxia/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


####  Import Library 📦

In [12]:
# Importation de packages nécessaires

import pandas as pd
import json
import nltk
import spacy
import matplotlib.pyplot as plt


from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer

## **1.** Création de la base de données des commentaires, tokenisation et lemmatisation

### **1.1** Création du DataFrame

Nous allons ouvrir le dataset de telle façon qu'une observation est un commentaire sur un film

In [14]:
# Transformation du fichier json en un Dataframe Pandas

# Charger le fichier JSON
with open("data_reviews_1o2.json", 'r') as file:
    data = json.load(file)

In [15]:
movie_name = []
comments = []
notes = []
year= []
budget=[]
recette=[]
duree=[]

for movie, dico in data.items():
    if '0' in dico and isinstance(dico['0'], list):  # Vérifier si la clé '0' est une liste
        for comment in dico['0']:
            movie_name.append(movie)
            comments.append(comment)
            year.append(dico['1'])
            notes.append(dico['2'])
            budget.append(dico['3'])
            recette.append(dico['4'])
            duree.append(dico['5'])

df = pd.DataFrame({
    'Film': movie_name,
    'Commentaire': comments,
    'Annee':year,
    'Note imdb': notes,
    'Budget':budget,
    'Box office':recette,
    'duree':duree
})

df.sample(5)

,Film,Commentaire,Annee,Note imdb,Budget,Box office,duree
17471,Les Animaux fantastiques (2016),Many reviews here are from Harry Potter fans w...,2016,"7,2",180000000,816037575,2h 12m
16437,Inception (2010),Not much more to add really. I managed to watc...,2010,"8,8",160000000,839030630,2h 28m
4959,Le Seigneur des anneaux : Le Retour du roi (2003),This movie could really use an EE -- BADLY. Ch...,2003,"9,0",94000000,1155870721,3h 21m
3362,Star Wars : Épisode VIII - Les Derniers Jedi (...,"For film fanatics like myself, The Last Jedi i...",2017,"6,9",317000000,1334407706,2h 32m
15095,Black Panther: Wakanda Forever (2022),As the last film in the fourth phase of the Ma...,2022,"6,7",250000000,859208836,2h 41m


### **1.2** Nettoyage des commentaires

#### **1.2.1** Tokenisation et suppression de la ponctuation

Il faut executer la cellule ci-dessous pour pouvoir charger le modèle de la langue anglaise de scapy.

In [16]:
# Paramètre permettant d'afficher un commentaire même si il est très long
pd.set_option('display.max_colwidth', None)

In [17]:
# On retire les "\n" dans les commentaires
df["Commentaire"] = df["Commentaire"].str.replace('\n','')
df["Commentaire"].sample()

17860    "Shrek the Third" is funny and has big laughs throughout, and one gets the feel that this installment of the big green guy's story is completely unnecessary. The cryptic symptom of Shrek's demise is somewhat insidious. All the big laughs come from the supporting characters namely Pinocchio (Cody Cameron) and the Ginger Bread Man (Conrad Vernon). Justin Timberlake voices a great turn as Artie, heir to Far Far Away Throne. He has a great scene where he says, "Sometimes we all get in our own way
" This wonderful sentiment is so misplaced in this feeble attempt to empower faith in oneself. Also the movie should have heeded its own advice. "Shrek" the first was endearing in its message of seeing beyond appearances and finding the value of a person's or ogre's soul. "Shrek" also had a great sense of humor about life and itself. Apparently, the makers of "Shrek the Third" have forgotten this legacy. And there are too many involved. Chris Miller is director along with co-director Rama

In [18]:
pd.set_option('display.max_colwidth', 100)

In [19]:
# On commence par charger le modèle de la langue anglaise de Spacy (les commentaires sont en anglais)
nlp = spacy.load('en_core_web_sm')

# Fonction qui prend en argument un commentaire (chaîne de caractère) et qui retourne une liste des tokens sans les stopwords et la ponctuation
def tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return tokens

In [20]:
%%time
# On crée une nouvelle colonne avec la liste des tokens pour chaque commentaire
df['Tokens'] = df['Commentaire'].apply(tokenize)
df.sample(5)

CPU times: user 25min 44s, sys: 7.02 s, total: 25min 51s
Wall time: 25min 59s


,Film,Commentaire,Annee,Note imdb,Budget,Box office,duree,Tokens
11108,Le Hobbit: La Désolation de Smaug (2013),This must be the best movie of 2013. There is no movie that comes close to it recently. I must s...,2013,"7,8",225000000,959027992,2h 41m,"[best, movie, 2013, movie, comes, close, recently, enjoyed, enjoy, second, far, experience, seei..."
2550,Barbie (2023),"""I'm just so tired of watching myself and every single other woman tie herself into knots so tha...",2023,"7,0",100000000,1441788910,1h 54m,"[tired, watching, single, woman, tie, knots, people, like, true, doll, ""Bro, excited, look, beau..."
5240,Aquaman (2018),"I was proper looking forward to this fish man hybrid film but wow just wow, how did they get thi...",2018,"6,8",160000000,1157347433,2h 23m,"[proper, looking, forward, fish, man, hybrid, film, wow, wow, wrong, watch, tin, paint, drying, ..."
12028,Les Minions 2: Il était une fois Gru (2022),"I cannot express my love for this beautiful masterpiece of art. This brought me to tears, howlin...",2022,"6,5",80000000,939628210,1h 27m,"[express, love, beautiful, masterpiece, art, brought, tears, howling, laughter, depression, fina..."
9924,Jumanji : Bienvenue dans la jungle (2017),"A modern twist provides a decent afternoon filler, end climax scene might be a bit OTT, but film...",2017,"6,9",90000000,995339117,1h 59m,"[modern, twist, provides, decent, afternoon, filler, end, climax, scene, bit, OTT, film, watcher..."


Avec Spacy, l'algortihme s'exécute en près de  min pour l'ensemble de la base de données. Nous avons utilisé Spacy et non nltk pour la tokenization et la suppression des stopwords, puisque Spacy a un répertoire plus important de stopwords et a de meilleures performances pour les textes volumineux. Il est à noter que la fonction de tokenisation ne supprime pas les négations, donc l'analyse de sentiment ne sera pas biaisée.

#### **1.2.2** Lemmatisation des commentaires   
Nous allons désormais procéder à la lemmatisation des commentaires pour pouvoir allèger les algorithmes de NLP plus tard.

In [ ]:
# Fonction qui prend en argument une liste de tokens et qui retourne ces tokens lemmatisés
def lemm(tokens):
    # D'abord, on transforme la liste en doc Spacy
    tokens_as_doc = spacy.tokens.Doc(nlp.vocab, words=tokens)
    # Lemmatisation du doc 
    lemmatized = [token.lemma_ for token in tokens_as_doc]
    return lemmatized
# Le lemma_ de Spacy ne reconnait pas les tokens et renvoie des listes vides (Pourquoi ?)

lemmatizer = WordNetLemmatizer()

def lemm2(tokens):
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized



In [ ]:
# On crée une nouvelle colonne des tokens lemmatisés
df['Tokens lemmatisés'] = df['Tokens'].apply(lemm2)
df.sample(5)

### **1.3** Représentation des mots

Maintenant, visualisons pour certains commentaires les mots les plus représentés après nettoyage des commentaires.

Commençons d'abord par visualiser un premier nuage de mots d'un commentaire quelconque.

In [ ]:
# Fonction qui prend en argument une liste de tokens et qui retourne le nuage de mots correspondant.

def cloud(tokens):
    text = " ".join(tokens)
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
    return wordcloud

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(cloud(df['Tokens lemmatisés'][0]), interpolation='bilinear')
plt.axis("off")
plt.show()

Par exemple, pour le premier commentaire de la bdd, on peut dès à présent avoir une idée sur son avis vis à vis du film grâce à ce nuage de mots, de par la présence de mots tels que 'masterpiece, 'magnificent', 'perfection'... qui sont assez représentés. Néanmoins, comme nous avons utilisé nltk, le lemmatiseur ne reconnait pas les entités nommés telles que 'IMAX', 'India'... . 
La présence des mots concernants le temps comme "waiting", "time", "longest" nous indique que la durée du film est un sujet des commentaires et nous pourrons voir par la suite s'il existe une corrélation entre la durée du film et la polarité des commentaires.

## **2.** NLP et analyse des sentiments

Dans un premier temps nous allons appliquer les algorithmes de NLP sur les commentaires afin d'obtenir un score pour chaque commentaires puis nous allons modifier le dataset de telle sorte qu'une observation est un film.

### **2.1** application d'algorithmes d'analyses de sentiments

In [ ]:
# Convertit les commentaires en string.
def string_function(column):
    return column.str.lower()

df['Commentaire'] =  string_function(df["Commentaire"])

Nous allons comparer les différents algorithmes proposés, notamment ceux des modules TextBlob, nltk.

On va dans un premier temps utiliser le module TextBlob

In [ ]:
def analyze_sentiment_blob(tokens):
    text = ' '.join(tokens)
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [ ]:
# Applique la fonction à la colonne 'tokens_lemmatized' du DataFrame
df['sentiment_polarity_blob'] = df['Tokens lemmatisés'].apply(analyze_sentiment_blob)

On essaye maintenant avec la fonction SentimentIntensityAnalyzer de nltk

In [ ]:
def analyze_sentiment_nltk(tokens):
    text = ' '.join(tokens)
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)['compound']

In [ ]:
%%time
df['sentiment_polarity_nltk'] = df["Tokens lemmatisés"].apply(analyze_sentiment_nltk)

La cellule ci-dessus prend environ 2 min à s'exécuter

### **2.2** manipulation pour la création du nouveau dataset

Les variables sont sous forme de string. Nous changeons cela pour pourvoir faire des statistiques avec.

In [ ]:
df.dtypes

In [ ]:
#change le type de la colonne 'Note imdb' en float
df['Note imdb'] = df['Note imdb'].str.replace(',', '.').astype(float)
df['Annee'] = df['Annee'].astype(float)
df["Budget"] = df["Budget"].astype(float)
df['Box office'] = df["Box office"].astype(float)

On groupe les observations selon le film.

In [ ]:
grouped = df.groupby('Film')
df_grouped = grouped[["sentiment_polarity_nltk", "sentiment_polarity_blob",  'Note imdb', 'Annee', "Budget", "Box office"]].mean()

Intéressons nous à la corrélation des notes avec les scores moyens:

On va également ajouter la durée de chaque film dans le nouveau df "df_grouped":

In [ ]:
# Définir une fonction pour sélectionner une observation pour chaque groupe
def select_observation(group):
    # Vous pouvez personnaliser cette fonction pour choisir une observation spécifique,
    # par exemple, ici, nous choisissons la première observation de chaque groupe.
    return group.iloc[0]

# Appliquer la fonction à chaque groupe
duration_serie = grouped['duree'].apply(select_observation)
duration_serie.head()

In [ ]:
df_grouped = pd.merge(duration_serie, df_grouped , on='Film', how='inner')
df_grouped.head()

Il peut être interessant de regarder aussi la variance des scores:

In [ ]:
var_by_group = grouped[["sentiment_polarity_nltk", "sentiment_polarity_blob"]].var()

In [ ]:
# "Join" var_by_group et le df pour ajouter les variables de variance
df_grouped_2 = pd.merge(var_by_group, df_grouped , on='Film', how='inner')

In [ ]:
# Renomme les variables obtenues:
df_grouped_2.rename(columns={'sentiment_polarity_nltk_x': 'var_score_nltk'}, inplace=True)
df_grouped_2.rename(columns={'sentiment_polarity_blob_x': 'var_score_blob'}, inplace=True)
df_grouped_2.rename(columns={'sentiment_polarity_nltk_y': 'sentiment_polarity_nltk'}, inplace=True)
df_grouped_2.rename(columns={'sentiment_polarity_blob_y': 'sentiment_polarity_blob'}, inplace=True)

# Rétablir l'index par défaut
df_grouped_2.reset_index(inplace=True)
print(df_grouped_2.shape)
df_grouped_2.head()

In [ ]:
df_grouped_2.to_csv('data_post_process_2o2.csv', index=False)

In [ ]:
df_grouped_2 = pd.read_csv('data_post_process_2o2.csv')

In [ ]:
#vérification
df_grouped_2.head()

Maintenant que nos data ont été traité, nous allons procéder à l'analyse statistique sur un autre notebook ("Analyse_modelisation_3o3") avec les "data_post_process_2o2" afin de faciliter la lecture du projet. En effet, l'exécution de ce notebook peut prendre une quinzaine de minute.